In [ ]:
import pandas as pd
from openprompt.data_utils import InputExample

# ==============================
# Load WNLI dataset
# ==============================
df = pd.read_csv(
    r"C:\Users\stdFurqan\Desktop\paft\WNLI\WNLI_train_urdu_entailment.csv"
)

print("Columns:", df.columns.tolist())

# ==============================
# WNLI Label Mapping (BINARY)
# ==============================
label_map = {
    "entailment": 1,
    "not_entailment": 0
}

classes = list(label_map.keys())

guid = 0
examples = []
class_stats = {}

# ==============================
# Take FIRST 16 examples per class
# ==============================
for label_name in classes:
    class_df = df[df["label_text"] == label_name].head(16)
    class_stats[label_name] = len(class_df)

    for _, row in class_df.iterrows():
        examples.append(
            InputExample(
                guid=guid,
                text_a=str(row["sentence1_urdu"]).strip(),
                text_b=str(row["sentence2_urdu"]).strip(),
                label=label_map[label_name]
            )
        )
        guid += 1

# ==============================
# Print InputExamples (exact format)
# ==============================
print("\n### InputExamples ###\n")
for ex in examples:
    print(
        "InputExample(\n"
        f"    guid={ex.guid},\n"
        f"    text_a=\"{ex.text_a}\",\n"
        f"    text_b=\"{ex.text_b}\",\n"
        f"    label={ex.label}\n"
        "),"
    )

# ==============================
# Metadata
# ==============================
print("\n### Classes ###")
print(f"classes = {classes}")

print("\n### Label Map ###")
print(f"label_map = {label_map}")

print("\n### Samples per Class ###")
for cls, count in class_stats.items():
    print(f"{cls}: {count}")

print(f"\nTotal InputExamples generated: {guid}")


In [ ]:
templates = [
    ("P1", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P2", ManualTemplate(
        text='پہلا بیان: {"placeholder":"text_a"} دوسرا بیان: {"placeholder":"text_b"} ان کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P3", ManualTemplate(
        text='{"placeholder":"text_b"} کیا {"placeholder":"text_a"} سے {"mask"} ہوتا ہے؟',
        tokenizer=tokenizer,
    )),

    ("P4", ManualTemplate(
        text='{"placeholder":"text_a"} کی روشنی میں {"placeholder":"text_b"} {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P5", ManualTemplate(
        text='{"placeholder":"text_b"} کا بیان {"placeholder":"text_a"} کے مطابق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P6", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے درمیان منطقی رشتہ {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P7", ManualTemplate(
        text='{"placeholder":"text_b"}، {"placeholder":"text_a"} سے {"mask"} طور پر جڑا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P8", ManualTemplate(
        text='اگر ہم {"placeholder":"text_a"} کو دیکھیں تو {"placeholder":"text_b"} {"mask"} بنتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P9", ManualTemplate(
        text='{"placeholder":"text_a"} کے حوالے سے {"placeholder":"text_b"} {"mask"} سمجھا جاتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} میں تعلق کی نوعیت {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
]


In [ ]:
verbalizer = ManualVerbalizer(
    classes=["not_entailment", "entailment"],
    label_words={
        "entailment": ["درست", "ثابت"],
        "not_entailment": ["غلط", "نامطابق"]
    },
    tokenizer=tokenizer,
)


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [3]:
# ==============================
# Balanced Batch Sampler
# ==============================
class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        """
        dataset: list of InputExample
        batch_size: total batch size (must be divisible by number of classes)
        """
        self.dataset = dataset
        self.labels = [ex.label for ex in dataset]
        self.classes = list(sorted(set(self.labels)))
        self.num_classes = len(self.classes)
        assert batch_size % self.num_classes == 0, "Batch size must be divisible by number of classes"
        self.batch_size_per_class = batch_size // self.num_classes

    def __iter__(self):
        class_indices = {c: np.where(np.array(self.labels) == c)[0].tolist() for c in self.classes}
        for c in self.classes:
            np.random.shuffle(class_indices[c])

        num_batches = min(len(class_indices[c]) // self.batch_size_per_class for c in self.classes)

        for i in range(num_batches):
            batch = []
            for c in self.classes:
                start = i * self.batch_size_per_class
                end = start + self.batch_size_per_class
                batch.extend(class_indices[c][start:end])
            np.random.shuffle(batch)
            yield batch

    def __len__(self):
        return min(len(np.where(np.array(self.labels) == c)[0]) // self.batch_size_per_class for c in self.classes)

In [4]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [

        InputExample(
            guid=0,
            text_a="ہنری کئی بار ان انٹرویوز میں موجود رہا تھا جو اس کے والد نے معروف جاسوسوں کے ساتھ کیے تھے جو پیچیدہ اسرار کو حل کرنے میں اس کی مدد چاہتے تھے اور وہ مواقع اس کے لیے سرخ حروف کے دنوں کے طور پر کھڑے تھے ۔",
            text_b="وہ مواقع ہنری کے لیے سرخ حروف والے دنوں کے طور پر نمایاں تھے ۔",
            label=1
        ),
        InputExample(
            guid=1,
            text_a="آدم اس وقت تک یہاں سے کام نہیں چھوڑ سکتا جب تک کہ باب اس کی جگہ لینے نہیں آتا ۔ اگر باب وقت پر کام کے لیے گھر سے نکل جاتا تو وہ اس وقت تک چلا جاتا ۔",
            text_b="آدم اس وقت تک چلا جائے گا ۔",
            label=1
        ),
        InputExample(
            guid=2,
            text_a="آدم اس وقت تک یہاں سے کام نہیں چھوڑ سکتا جب تک کہ باب اس کی جگہ لینے نہیں آتا ۔ اگر باب وقت پر کام کے لیے گھر سے نکل جاتا تو وہ اس وقت تک یہاں پہنچ جاتا ۔",
            text_b="باب اس وقت تک یہاں پہنچ چکا ہوگا ۔",
            label=1
        ),
        InputExample(
            guid=3,
            text_a="آدم اس وقت تک یہاں سے کام نہیں چھوڑ سکتا جب تک کہ باب اس کی جگہ لینے نہیں آتا ۔ اگر باب وقت پر کام کے لیے گھر سے نکل جاتا تو وہ اس وقت تک یہاں پہنچ جاتا ۔",
            text_b="باب اس وقت تک یہاں پہنچ چکا ہوگا ۔",
            label=1
        ),
        InputExample(
            guid=4,
            text_a="ایلس نے ہجوم میں اپنی دوست جیڈ کو تلاش کیا ۔ چونکہ وہ ہمیشہ خوش قسمت رہتی ہے ۔ ایلس نے اسے جلدی سے دیکھ لیا ۔",
            text_b="چونکہ ایلس ہمیشہ خوش قسمت رہتی ہے ، اس لیے ایلس نے اسے جلدی سے دیکھ لیا ۔",
            label=1
        ),
        InputExample(
            guid=5,
            text_a="ایلس نے ہجوم میں اپنی دوست جیڈ کو تلاش کیا ۔ چونکہ وہ ہمیشہ سرخ پگڑی پہنتی ہے اس لیے ایلس نے اسے جلدی سے دیکھ لیا ۔",
            text_b="چونکہ جیڈ ہمیشہ سرخ پگڑی پہنتی ہے ، اس لیے ایلس نے اسے جلدی سے دیکھا ۔",
            label=1
        ),
        InputExample(
            guid=6,
            text_a="ایلس نے ہجوم میں اپنی دوست جیڈ کو تلاش کیا ۔ چونکہ وہ ہمیشہ سرخ پگڑی پہنتی ہے اس لیے ایلس نے اسے جلدی سے دیکھ لیا ۔",
            text_b="چونکہ جیڈ ہمیشہ سرخ پگڑی پہنتی ہے ، اس لیے ایلس نے اسے جلدی سے دیکھا ۔",
            label=1
        ),
        InputExample(
            guid=7,
            text_a="ایلس نے پارٹی میں اپنی بیٹی کو بھونکنے سے روکنے کی بے چینی سے کوشش کی، جس سے ہم یہ سوچنے پر مجبور ہو گئے کہ وہ اتنا عجیب برتاؤ کیوں کر رہی تھی۔",
            text_b="ایلس کی بیٹی بہت عجیب سلوک کر رہی تھی ۔",
            label=1
        ),
        InputExample(
            guid=8,
            text_a="ایلس نے گھبراہٹ میں اپنی بیٹی کو پارٹی میں باتیں کرنے سے روکنے کی کوشش کی، جس سے ہمیں یہ سوچنے پر مجبور ہونا پڑا کہ وہ اتنا عجیب رویہ کیوں اختیار کر رہی تھی۔",
            text_b="ایلس بہت عجیب سلوک کر رہی تھی ۔",
            label=1
        ),
        InputExample(
            guid=9,
            text_a="ایلس ڈرائنگ روم کی گرد جھاڑ رہی تھی اور وہ بٹن ڈھونڈنے کی کوشش کر رہی تھی جو ماں نے چھپا دیا تھا۔ آج اس کے پاس اپنی پسندیدہ فوٹو البم میں پرانی تصویریں دیکھنے کا وقت نہیں تھا۔ آج اسے ایک بٹن تلاش کرنا تھا، اس لیے اس نے البم کو بغیر کھولے ہی ایک کرسی پر رکھ دیا۔",
            text_b="ایلس کے پسندیدہ فوٹو البم میں پرانی تصاویر دیکھنے کے لیے آج وقت نہیں ہے ۔",
            label=1
        ),
        InputExample(
            guid=10,
            text_a="ایلس ڈرائنگ روم کی گرد جھاڑ رہی تھی اور وہ بٹن ڈھونڈنے کی کوشش کر رہی تھی جو ماں نے چھپا دیا تھا۔ آج اس کے پاس اپنی پسندیدہ فوٹو البم میں پرانی تصویریں دیکھنے کا وقت نہیں تھا۔ آج اسے ایک بٹن تلاش کرنا تھا، اس لیے اس نے البم کو بغیر کھولے ہی ایک کرسی پر رکھ دیا۔",
            text_b="اس نے البم کو کھولے بغیر ہی اسے کرسی پر رکھ دیا ۔",
            label=1
        ),
        InputExample(
            guid=11,
            text_a="ایلس ڈرائنگ روم کی گرد جھاڑ رہی تھی اور وہ بٹن ڈھونڈنے کی کوشش کر رہی تھی جو ماں نے چھپا دیا تھا۔ آج اس کے پاس اپنی پسندیدہ فوٹو البم میں پرانی تصویریں دیکھنے کا وقت نہیں تھا۔ آج اسے ایک بٹن تلاش کرنا تھا، اس لیے اس نے البم کو بغیر کھولے ہی ایک کرسی پر رکھ دیا۔",
            text_b="ایلس کو ایک بٹن تلاش کرنا پڑا ۔",
            label=1
        ),
        InputExample(
            guid=12,
            text_a="ڈورا کے چیک دار لباس کی پشت پر لگے تمام بٹن الٹے رخ سے بند کیے گئے تھے۔ ماوڈ کو چاہیے تھا کہ وہ اسے خود بٹن لگا دیتی، مگر نہیں؛ اس نے بیچاری ننھی ڈورا کو اکیلا ہی اپنی بساط کے مطابق سب کچھ کرنے کے لیے چھوڑ دیا تھا۔",
            text_b="اس نے بیچاری ننھی ڈورا کو اکیلا ہی چھوڑ دیا تھا کہ وہ اپنی بساط کے مطابق خود ہی سب کچھ کر لے۔",
            label=1
        ),
        InputExample(
            guid=13,
            text_a="اگرچہ وہ تقریبا ایک ہی رفتار سے دوڑتے تھے ۔ سو نے سیلی کو شکست دی کیونکہ اس کی شروعات اتنی خراب تھی ۔",
            text_b="سیلی کی شروعات اتنی خراب تھی ۔",
            label=1
        ),
        InputExample(
            guid=14,
            text_a="اگرچہ وہ تقریبا ایک ہی رفتار سے دوڑتے تھے ۔ سو نے سیلی کو شکست دی کیونکہ اس نے اتنی اچھی شروعات کی تھی ۔",
            text_b="سو کی شروعات اچھی رہی ۔",
            label=1
        ),
        InputExample(
            guid=15,
            text_a="ہمیشہ پہلے لیری نے والد کو ان کے کام میں مدد کی تھی ۔ لیکن وہ اب ان کی مدد نہیں کر سکے کیونکہ والد نے کہا کہ ریلوے کمپنی میں ان کے باس نہیں چاہیں گے کہ ان کے علاوہ کوئی دفتر میں کام کرے ۔",
            text_b="وہ اب والد کی مدد نہیں کر سکتا تھا ۔",
            label=1
        ),
        InputExample(
            guid=16,
            text_a="ہنری کئی بار ان انٹرویوز میں موجود رہا تھا جو اس کے والد نے معروف جاسوسوں کے ساتھ کیے تھے جو پیچیدہ اسرار کو حل کرنے میں اس کی مدد چاہتے تھے اور وہ مواقع اس کے لیے سرخ حروف کے دنوں کے طور پر کھڑے تھے ۔",
            text_b="وہ مواقع والد کے لیے یادگار اور خاص دنوں کی حیثیت رکھتے تھے۔",
            label=0
        ),
        InputExample(
            guid=17,
            text_a="آدم اس وقت تک یہاں سے کام نہیں چھوڑ سکتا جب تک کہ باب اس کی جگہ لینے نہیں آتا ۔ اگر باب وقت پر کام کے لیے گھر سے نکل جاتا تو وہ اس وقت تک چلا جاتا ۔",
            text_b="باب اس وقت تک چلا جائے گا ۔",
            label=0
        ),
        InputExample(
            guid=18,
            text_a="آدم اس وقت تک یہاں سے کام نہیں چھوڑ سکتا جب تک کہ باب اس کی جگہ لینے نہیں آتا ۔ اگر باب وقت پر کام کے لیے گھر سے نکل جاتا تو وہ اس وقت تک یہاں پہنچ جاتا ۔",
            text_b="آدم اس وقت تک یہاں پہنچ چکا ہوگا ۔",
            label=0
        ),
        InputExample(
            guid=19,
            text_a="ایلس نے ہجوم میں اپنی دوست جیڈ کو تلاش کیا ۔ چونکہ وہ ہمیشہ خوش قسمت رہتی ہے ۔ ایلس نے اسے جلدی سے دیکھ لیا ۔",
            text_b="چونکہ جیڈ ہمیشہ خوش قسمت رہتی ہے ، اس لیے ایلس نے اسے جلدی سے دیکھ لیا ۔",
            label=0
        ),
        InputExample(
            guid=20,
            text_a="ایلس نے پارٹی میں اپنی بیٹی کو بھونکنے سے روکنے کی بے چینی سے کوشش کی، جس سے ہم یہ سوچنے پر مجبور ہو گئے کہ وہ اتنا عجیب برتاؤ کیوں کر رہی تھی۔",
            text_b="ایلس بہت عجیب سلوک کر رہی تھی ۔",
            label=0
        ),
        InputExample(
            guid=21,
            text_a="ایلس نے گھبراہٹ میں اپنی بیٹی کو پارٹی میں باتیں کرنے سے روکنے کی کوشش کی، جس سے ہمیں یہ سوچنے پر مجبور ہونا پڑا کہ وہ اتنا عجیب رویہ کیوں اختیار کر رہی تھی۔",
            text_b="ایلس کی بیٹی بہت عجیب سلوک کر رہی تھی ۔",
            label=0
        ),
        InputExample(
            guid=22,
            text_a="ایلس نے گھبراہٹ میں اپنی بیٹی کو پارٹی میں باتیں کرنے سے روکنے کی کوشش کی، جس سے ہمیں یہ سوچنے پر مجبور ہونا پڑا کہ وہ اتنا عجیب رویہ کیوں اختیار کر رہی تھی۔",
            text_b="ایلس کی بیٹی بہت عجیب سلوک کر رہی تھی ۔",
            label=0
        ),
        InputExample(
            guid=23,
            text_a="ایلس ڈرائنگ روم کی گرد جھاڑ رہی تھی اور وہ بٹن ڈھونڈنے کی کوشش کر رہی تھی جو ماں نے چھپا دیا تھا۔ آج اس کے پاس اپنی پسندیدہ فوٹو البم میں پرانی تصویریں دیکھنے کا وقت نہیں تھا۔ آج اسے ایک بٹن تلاش کرنا تھا، اس لیے اس نے البم کو بغیر کھولے ہی ایک کرسی پر رکھ دیا۔",
            text_b="اس نے رہنے کا کمرہ کھولے بغیر بھی البم کو کرسی پر رکھ دیا ۔",
            label=0
        ),
        InputExample(
            guid=24,
            text_a="ایلس ڈرائنگ روم کی گرد جھاڑ رہی تھی اور وہ بٹن ڈھونڈنے کی کوشش کر رہی تھی جو ماں نے چھپا دیا تھا۔ آج اس کے پاس اپنی پسندیدہ فوٹو البم میں پرانی تصویریں دیکھنے کا وقت نہیں تھا۔ آج اسے ایک بٹن تلاش کرنا تھا، اس لیے اس نے البم کو بغیر کھولے ہی ایک کرسی پر رکھ دیا۔",
            text_b="ماما کو بٹن تلاش کرنا پڑا ۔",
            label=0
        ),
        InputExample(
            guid=25,
            text_a="ایلس ڈرائنگ روم کی گرد جھاڑ رہی تھی اور وہ بٹن ڈھونڈنے کی کوشش کر رہی تھی جو ماں نے چھپا دیا تھا۔ آج اس کے پاس اپنی پسندیدہ فوٹو البم میں پرانی تصویریں دیکھنے کا وقت نہیں تھا۔ آج اسے ایک بٹن تلاش کرنا تھا، اس لیے اس نے البم کو بغیر کھولے ہی ایک کرسی پر رکھ دیا۔",
            text_b="آج ماما کے پسندیدہ فوٹو البم میں پرانی تصاویر دیکھنے کا وقت نہیں ہے ۔",
            label=0
        ),
        InputExample(
            guid=26,
            text_a="ایلس ڈرائنگ روم کی گرد جھاڑ رہی تھی اور وہ بٹن ڈھونڈنے کی کوشش کر رہی تھی جو ماں نے چھپا دیا تھا۔ آج اس کے پاس اپنی پسندیدہ فوٹو البم میں پرانی تصویریں دیکھنے کا وقت نہیں تھا۔ آج اسے ایک بٹن تلاش کرنا تھا، اس لیے اس نے البم کو بغیر کھولے ہی ایک کرسی پر رکھ دیا۔",
            text_b="اس نے کرسی کھولے بغیر البم کو کرسی پر رکھ دیا ۔",
            label=0
        ),
        InputExample(
            guid=27,
            text_a="ایلس ڈرائنگ روم کی گرد جھاڑ رہی تھی اور وہ بٹن ڈھونڈنے کی کوشش کر رہی تھی جو ماں نے چھپا دیا تھا۔ آج اس کے پاس اپنی پسندیدہ فوٹو البم میں پرانی تصویریں دیکھنے کا وقت نہیں تھا۔ آج اسے ایک بٹن تلاش کرنا تھا، اس لیے اس نے البم کو بغیر کھولے ہی ایک کرسی پر رکھ دیا۔",
            text_b="اس نے بٹن کھولے بغیر البم کو کرسی پر رکھ دیا ۔",
            label=0
        ),
        InputExample(
            guid=28,
            text_a="ڈورا کے چیک دار لباس کی پشت پر لگے تمام بٹن الٹے رخ سے بند کیے گئے تھے۔ ماوڈ کو چاہیے تھا کہ وہ اسے خود بٹن لگا دیتی، مگر نہیں؛ اس نے بیچاری ننھی ڈورا کو اکیلا ہی اپنی بساط کے مطابق سب کچھ کرنے کے لیے چھوڑ دیا تھا۔",
            text_b="اس نے بےچاری ننھی ڈورا کو اکیلا چھوڑ دیا تھا کہ وہ اپنی پوری کوشش خود ہی کرے۔",
            label=0
        ),
        InputExample(
            guid=29,
            text_a="اگرچہ وہ تقریبا ایک ہی رفتار سے دوڑتے تھے ۔ سو نے سیلی کو شکست دی کیونکہ اس کی شروعات اتنی خراب تھی ۔",
            text_b="سو کی شروعات اتنی خراب تھی ۔",
            label=0
        ),
        InputExample(
            guid=30,
            text_a="ہمیشہ پہلے لیری نے والد کو ان کے کام میں مدد کی تھی ۔ لیکن وہ اب ان کی مدد نہیں کر سکے کیونکہ والد نے کہا کہ ریلوے کمپنی میں ان کے باس نہیں چاہیں گے کہ ان کے علاوہ کوئی دفتر میں کام کرے ۔",
            text_b="بابا اب اس کی مدد نہیں کر سکتے تھے ۔",
            label=0
        ),
        InputExample(
            guid=31,
            text_a="ہمیشہ پہلے لیری نے والد کو ان کے کام میں مدد کی تھی ۔ لیکن وہ اب ان کی مدد نہیں کر سکے کیونکہ والد نے کہا کہ ریلوے کمپنی میں ان کے باس نہیں چاہیں گے کہ ان کے علاوہ کوئی دفتر میں کام کرے ۔",
            text_b="ریلوے کمپنی میں لیری کا باس نہیں چاہتا تھا کہ اس کے علاوہ کوئی بھی دفتر میں کام کرے ۔",
            label=0
        ),
        
    
]


In [5]:
### Classes ###
classes = ['entailment', 'not_entailment']

### Label Map ###
label_map = {'entailment': 1, 'not_entailment': 0}



# # # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
# _, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # # Step 2: Manually load XLM-RoBERTa model/tokenizer
# model_name = "xlm-roberta-base"
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
# plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized fr

In [6]:
# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    text = '{"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
    tokenizer=tokenizer,
)




templates = [
    ("P1", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P2", ManualTemplate(
        text='پہلا بیان: {"placeholder":"text_a"} دوسرا بیان: {"placeholder":"text_b"} ان کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P3", ManualTemplate(
        text='{"placeholder":"text_b"} کیا {"placeholder":"text_a"} سے {"mask"} ہوتا ہے؟',
        tokenizer=tokenizer,
    )),

    ("P4", ManualTemplate(
        text='{"placeholder":"text_a"} کی روشنی میں {"placeholder":"text_b"} {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P5", ManualTemplate(
        text='{"placeholder":"text_b"} کا بیان {"placeholder":"text_a"} کے مطابق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P6", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے درمیان منطقی رشتہ {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P7", ManualTemplate(
        text='{"placeholder":"text_b"}، {"placeholder":"text_a"} سے {"mask"} طور پر جڑا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P8", ManualTemplate(
        text='اگر ہم {"placeholder":"text_a"} کو دیکھیں تو {"placeholder":"text_b"} {"mask"} بنتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P9", ManualTemplate(
        text='{"placeholder":"text_a"} کے حوالے سے {"placeholder":"text_b"} {"mask"} سمجھا جاتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} میں تعلق کی نوعیت {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
]







verbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
        "entailment": ["درست", "ثابت"],
        "not_entailment": ["غلط", "نامطابق"]
    },
    tokenizer=tokenizer,
)

In [7]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# Training loop with BalancedBatchSampler + random template switching
# ==============================
T = 20   # epochs
K = 1    # steps per prompt
batch_size = 16





prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)
all_epoch_patterns = {}

for epoch in range(T):
    print(f"\n🟦 Epoch {epoch+1}/{T}")

    # Random initial template
    prompt_name, current_template = random.choice(templates)
    epoch_pattern = []

    # Create PromptDataLoader with BalancedBatchSampler
    sampler = BalancedBatchSampler(train_dataset, batch_size=batch_size)
    train_loader = PromptDataLoader(
        dataset=train_dataset,
        tokenizer=tokenizer,
        template=current_template,
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=128,
        batch_size=batch_size,
        batch_sampler=sampler,
        shuffle=False  # shuffle is ignored when batch_sampler is used
    )

    step_counter = 0
    epoch_loss = 0.0

    for batch in train_loader:
        # Move batch to device
        # batch = {k: v.to(device) for k, v in batch.items()}
    
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        # loss = criterion(logits, batch["label"])
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_pattern.append(prompt_name)

        step_counter += 1

        # Switch template every K steps
        if step_counter % K == 0:
            prompt_name, current_template = random.choice(templates)

            # Rebuild PromptDataLoader with new template but same sampler
            train_loader = PromptDataLoader(
                dataset=train_dataset,
                tokenizer=tokenizer,
                template=current_template,
                tokenizer_wrapper_class=WrapperClass,
                max_seq_length=128,
                batch_size=batch_size,
                batch_sampler=sampler,
                shuffle=False
            )

    all_epoch_patterns[f"epoch_{epoch+1}"] = epoch_pattern
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
    print(f"Prompt pattern: {epoch_pattern}")


🟦 Epoch 1/20


tokenizing: 32it [00:00, 508.10it/s]
tokenizing: 32it [00:00, 843.61it/s]
tokenizing: 32it [00:00, 793.50it/s]


Epoch 1 Loss: 5.5924
Prompt pattern: ['P2', 'P1']

🟦 Epoch 2/20


tokenizing: 32it [00:00, 841.62it/s]
tokenizing: 32it [00:00, 873.42it/s]
tokenizing: 32it [00:00, 735.01it/s]


Epoch 2 Loss: 2.8597
Prompt pattern: ['P4', 'P4']

🟦 Epoch 3/20


...

Epoch 18 Loss: 1.7202
Prompt pattern: ['P10', 'P6']

🟦 Epoch 19/20


tokenizing: 32it [00:00, 945.72it/s]
tokenizing: 32it [00:00, 749.48it/s]
tokenizing: 32it [00:00, 950.97it/s]


Epoch 19 Loss: 1.1938
Prompt pattern: ['P4', 'P2']

🟦 Epoch 20/20


tokenizing: 32it [00:00, 746.78it/s]
tokenizing: 32it [00:00, 893.64it/s]
tokenizing: 32it [00:00, 850.85it/s]

Epoch 20 Loss: 1.2499
Prompt pattern: ['P4', 'P5']


In [8]:
# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(
    r"C:\Users\stdFurqan\Desktop\paft\WNLI\WNLI_dev_urdu_entailment.csv"
)

# Make InputExamples
eval_dataset = [
    InputExample(
        guid=i,
        text_a=row['Sentence1'],
        text_b=row['Sentence2'],
        label=label_map[row['label_text']]
    )
    for i, row in df.iterrows()
]

# ==============================
# PromptDataLoader
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,              # your ManualTemplate for XNLI
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 WNLI Urdu Dev Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))

tokenizing: 71it [00:00, 1171.57it/s]



📊 WNLI Urdu Dev Classification Report:
                precision    recall  f1-score   support

    entailment     0.5185    0.3500    0.4179        40
not_entailment     0.4091    0.5806    0.4800        31

      accuracy                         0.4507        71
     macro avg     0.4638    0.4653    0.4490        71
  weighted avg     0.4707    0.4507    0.4450        71



In [ ]:
roberta
BS = 8
📊 WNLI Urdu Dev Classification Report:
                precision    recall  f1-score   support

    entailment     0.6667    0.0500    0.0930        40
not_entailment     0.4412    0.9677    0.6061        31

      accuracy                         0.4507        71
     macro avg     0.5539    0.5089    0.3495        71
  weighted avg     0.5682    0.4507    0.3170        71



BS = 16

📊 WNLI Urdu Dev Classification Report:
                precision    recall  f1-score   support

    entailment     0.8571    0.1500    0.2553        40
not_entailment     0.4688    0.9677    0.6316        31

      accuracy                         0.5070        71
     macro avg     0.6629    0.5589    0.4434        71
  weighted avg     0.6876    0.5070    0.4196        71


In [ ]:
bert
BS = 8
📊 WNLI Urdu Dev Classification Report:
                precision    recall  f1-score   support

    entailment     0.6429    0.4500    0.5294        40
not_entailment     0.4884    0.6774    0.5676        31

      accuracy                         0.5493        71
     macro avg     0.5656    0.5637    0.5485        71
  weighted avg     0.5754    0.5493    0.5461        71

BS = 16
📊 WNLI Urdu Dev Classification Report:
                precision    recall  f1-score   support

    entailment     0.5185    0.3500    0.4179        40
not_entailment     0.4091    0.5806    0.4800        31

      accuracy                         0.4507        71
     macro avg     0.4638    0.4653    0.4490        71
  weighted avg     0.4707    0.4507    0.4450        71